# T1 SEMANTIC SEGMENTATION TRAINING

Small example on how to train Pointcept for semantic segmentation

## LIBRARIES

In [1]:
#IMPORT PACKAGES
from pathlib import Path
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('.')))
sys.path.insert(0, os.path.abspath(os.path.join('scripts')))
sys.path.insert(0, os.path.abspath(os.path.join('thirdparty', 'pointcept')))
print(sys.path)
import numpy as np
import laspy
import geomapi
from geomapi.utils import geometryutils as gmu
# import geomapi.utils as ut
import torch

import context 
import utils.t1_utils as t1
import utils as ut

# #MULTIPROCESSING -> didn't help setting this to spawn
# import multiprocessing as mp
# mp.set_start_method('spawn')  # or 'forkserver'


#POINTCEPT
from pointcept.engines.defaults import (
    default_argument_parser,
    default_config_parser,
    default_setup,
)
from pointcept.engines.train import TRAINERS
from pointcept.engines.launch import launch

['/home/mbassier/code/Scan-to-BIM-CVPR-2024/scripts/thirdparty/pointcept', '/home/mbassier/code/Scan-to-BIM-CVPR-2024/scripts/scripts', '/home/mbassier/code/Scan-to-BIM-CVPR-2024/scripts', '/home/mbassier/code/Scan-to-BIM-CVPR-2024/scripts', '/home/mbassier/.conda/envs/pointcept/lib/python310.zip', '/home/mbassier/.conda/envs/pointcept/lib/python3.10', '/home/mbassier/.conda/envs/pointcept/lib/python3.10/lib-dynload', '', '/home/mbassier/.local/lib/python3.10/site-packages', '/home/mbassier/.conda/envs/pointcept/lib/python3.10/site-packages', '/home/mbassier/.conda/envs/pointcept/lib/python3.10/site-packages/pointops-1.0-py3.10-linux-x86_64.egg', '/home/mbassier/.conda/envs/pointcept/lib/python3.10/site-packages/pointgroup_ops-0.0.0-py3.10-linux-x86_64.egg']
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
torch.cuda.empty_cache()  # Clear unused memory

In [4]:
%load_ext autoreload

In [5]:
%autoreload 2

tensorboard --logdir=/home/mbassier/code/Scan-to-BIM-CVPR-2024/data/t1_data_test/


## INPUT DATA CONVERSION

create config file

In [2]:
data_root= Path(os.getcwd()).parents[0]/'data'/'t1'

#PREPROCESSING
input_folder = data_root/'input'
training_folder = data_root/'train'
validation_folder = data_root/'val'

#TRAINING
config_path = data_root/'config.py' #Path(os.getcwd())/'data'/'t1_data'/'config.py'
# save_path = data_root #Path(os.getcwd())/'data'/'t1_data'
# weights =  #Path(os.getcwd())/'data'/'t1_data'/'model'/'model_best.pth'


cfg = default_config_parser(str(config_path), {'save_path': str(data_root), 
                                               'weight': str(data_root/'model'/'model_best.pth'),
                                               'data_root': str(data_root),})
# cfg = default_config_parser(str(config_path), {})
# cfg['data_root'] = f'({str(data_root)}, )'
cfg['data']['train']['data_root'] = str(data_root)
cfg['data']['val']['data_root'] = str(data_root)
cfg['data']['test']['data_root'] = str(data_root)
cfg['data']['names']=['floors', 'ceiling', 'wall', 'column', 'door','unassigned'] 
cfg['data']['labels']=[0,1,2,3,4,255] 
cfg['data']['num_classes']=6
cfg['data']['ignore_index']=-1
cfg['num_worker']=1


Preprocessing of input data

In [7]:
# # file=[n for n in ut.get_list_of_files(input_folder, '.laz') if 'normals' in n][0]
# las = laspy.read('/home/mbassier/code/Scan-to-BIM-CVPR-2024/data/t1/input/05_MedOffice_01_F2_pcdFLtrain.laz')

# print(list(las.point_format.dimension_names))
# print(np.unique(las['classes']))


In [10]:
training_folder=cfg['data']['train']['data_root']
validation_folder=cfg['data']['val']['data_root']

os.makedirs(training_folder, exist_ok=True)
os.makedirs(validation_folder, exist_ok=True)

[t1.handle_process(p, training_folder,cfg) for p in ut.get_list_of_files(input_folder, ext=".laz") if "train" in p]
[t1.handle_process(p, validation_folder,cfg) for p in ut.get_list_of_files(input_folder, ext=".laz") if "val" in p]

processing /home/mbassier/code/Scan-to-BIM-CVPR-2024/data/t1/input/05_MedOffice_01_F2_pcdFLtrain.laz ...
Function handle_process took 117.3973 seconds to execute.
processing /home/mbassier/code/Scan-to-BIM-CVPR-2024/data/t1/input/05_MedOffice_01_F2_pcdFLval.laz ...
Function handle_process took 135.6447 seconds to execute.


[None]

## TRAINING

Training using Point Transformer V3

In [3]:

def main_worker(cfg):
    cfg = default_setup(cfg)
    trainer = TRAINERS.build(dict(type=cfg.train.type, cfg=cfg))
    trainer.train()

launch(
    main_worker,
    num_gpus_per_machine=1, # these seems to be a proble with the multiprocessing -> first check spawn vs forkserver methods (we now use spawn, didn't resolve anything)
    num_machines=1,
    machine_rank=0,
    dist_url='auto',
    cfg=(cfg,),
)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/mbassier/.conda/envs/pointcept/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/mbassier/.conda/envs/pointcept/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'main_worker' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/mbassier/.conda/envs/pointcept/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/mbassier/.conda/envs/pointcept/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'main_worker' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>


ProcessExitedException: process 1 terminated with exit code 1